# Com UF, sem Contribuição

In [2]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
tcc_data = pd.read_csv("dadosTCC_final.csv")
tcc_data.head()

,Cliente,UF,Ano,Valor da Operação em R$,Valor Desembolsado em Reais,Setor CNAE,Subsetor CNAE Agrupado,Porte do cliente,Natureza,Instituição Financeira Credenciada,Contribuição do UF ao PIB Nacional (%),Proporção do Valor Setorial ao PIB do UF
0,DAMBROZ EMPREENDIMENTOS E ADMINISTRACAO LTDA,Rio Grande do Sul,2002,555255,555255.0,INDUSTRIA DE TRANSFORMAÇÃO,Máquinas e equipamentos,MÉDIA,PRIVADA,CHINA CONSTRUCTION BANK (BRASIL) BANCO MULTIPL...,0.066394,0.135932
1,COROL COOP AGROINDUSTRIAL,Paraná,2002,783125,783125.0,AGROPECUÁRIA E PESCA,Agropecuária,GRANDE,PRIVADA,CHINA CONSTRUCTION BANK (BRASIL) BANCO MULTIPL...,0.059267,0.053919
2,C S PESQUISAS E PARTICIPACOES INDUSTRIAIS LTDA,Paraná,2002,643001,644789.0,INDUSTRIA DE TRANSFORMAÇÃO,"Couro, artefato e calçado",GRANDE,PRIVADA,"BANCO BRADESCO S,A,",0.059267,0.152279
3,"RENUKA VALE DO IVAI S,A, - EM RECUPERACAO JUDI...",Paraná,2002,1549400,1713850.0,INDUSTRIA DE TRANSFORMAÇÃO,Produtos Alimentícios,GRANDE,PRIVADA,CHINA CONSTRUCTION BANK (BRASIL) BANCO MULTIPL...,0.059267,0.152279
4,FRIMESA COOPERATIVA CENTRAL,Paraná,2002,2721825,3290906.0,INDUSTRIA DE TRANSFORMAÇÃO,Produtos Alimentícios,MÉDIA,PRIVADA,"KIRTON BANK S,A, - BANCO MULTIPLO",0.059267,0.152279


In [4]:
# Carregar os dados
tcc_data = pd.read_csv("dadosTCC_final.csv")

# Preparar variáveis dependentes e independentes
dependent_var = tcc_data['Valor Desembolsado em Reais']
df_dummy = tcc_data.drop(columns=['Contribuição do UF ao PIB Nacional (%)', 'Proporção do Valor Setorial ao PIB do UF','Natureza', 'Ano', 'Cliente', 'Valor Desembolsado em Reais', 'Subsetor CNAE Agrupado', 'Instituição Financeira Credenciada', 'Valor da Operação em R$'])

# Criar variáveis dummy para dados categóricos
df_dummy = pd.get_dummies(df_dummy, drop_first=True)

# Remover qualquer linha com NaN e converter para tipo numérico
df_dummy = df_dummy.dropna().astype(float)

# Aplicar a transformação exponencial nas variáveis independentes
df_dummy = np.exp(df_dummy)  # Eleva e à potência dos valores em df_dummy

# Adicionar coluna constante para intercepto
df_dummy = sm.add_constant(df_dummy)  # Adiciona uma coluna constante para o intercepto

# Garantir que dependent_var está alinhada com df_dummy
tcc_data = tcc_data.loc[df_dummy.index]
dependent_var = tcc_data['Valor Desembolsado em Reais']

# Aplicar a transformação logarítmica na variável dependente, garantindo que não há valores zero ou negativos
dependent_var = np.log(dependent_var + 1)  # Usando log(x + 1) para lidar com valores zero

# Resetar índices para garantir alinhamento
dependent_var = dependent_var.reset_index(drop=True)
df_dummy = df_dummy.reset_index(drop=True)

# Criar o modelo de regressão OLS
model = sm.OLS(dependent_var, df_dummy)

# Ajustar o modelo
results = model.fit()

# Calcular a mudança percentual para cada coeficiente
coefficients = pd.DataFrame({
    'coef': results.params,
    'std err': results.bse,
    't': results.tvalues,
    'P>|t|': results.pvalues,
    'conf_int_low': results.conf_int()[0],
    'conf_int_high': results.conf_int()[1]
})
coefficients['percent_change'] = (np.exp(coefficients['coef']) - 1) * 100

# Salvando os resultados e a mudança percentual em um CSV
csv_path = 'log_regTCC_sem_Contribuicao_com_UF.csv'
coefficients.to_csv(csv_path, index_label='variable')
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:     Valor Desembolsado em Reais   R-squared:                       0.279
Model:                                     OLS   Adj. R-squared:                  0.277
Method:                          Least Squares   F-statistic:                     97.65
Date:                         Wed, 29 May 2024   Prob (F-statistic):               0.00
Time:                                 15:30:58   Log-Likelihood:                -11809.
No. Observations:                         7080   AIC:                         2.368e+04
Df Residuals:                             7051   BIC:                         2.387e+04
Df Model:                                   28                                         
Covariance Type:                     nonrobust                                         
=========================================================================================================
                                            coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------
const                                    27.2922     12.592      2.167      0.030       2.608      51.977
UF_Alagoas                               -0.2724      0.626     -0.435      0.663      -1.499       0.954
UF_Amazonas                               0.3392      0.543      0.624      0.533      -0.726       1.404
UF_Bahia                                 -0.0617      0.534     -0.116      0.908      -1.108       0.984
UF_Ceará                                 -0.3228      0.532     -0.607      0.544      -1.366       0.720
UF_Espírito Santo                        -0.1835      0.534     -0.343      0.731      -1.231       0.864
UF_Goiás                                 -0.5410      0.563     -0.960      0.337      -1.645       0.563
UF_Mato Grosso                           -0.6040      0.553     -1.093      0.274      -1.687       0.479
UF_Mato Grosso do Sul                    -0.7852      0.683     -1.150      0.250      -2.124       0.553
UF_Minas Gerais                           0.0854      0.530      0.161      0.872      -0.954       1.125
UF_Paraná                                -0.3389      0.530     -0.640      0.523      -1.378       0.700
UF_Paraíba                                0.0528      0.600      0.088      0.930      -1.123       1.229
UF_Pará                                  -0.8164      0.563     -1.450      0.147      -1.920       0.288
UF_Pernambuco                            -0.5591      0.535     -1.046      0.296      -1.607       0.489
UF_Piauí                                 -0.2733      0.592     -0.462      0.644      -1.434       0.887
UF_Rio Grande do Norte                   -0.4384      0.544     -0.806      0.421      -1.505       0.628
UF_Rio Grande do Sul                     -0.3123      0.529     -0.590      0.555      -1.350       0.725
UF_Rio de Janeiro                        -0.0067      0.532     -0.013      0.990      -1.051       1.037
UF_Rondônia                              -0.5503      0.579     -0.950      0.342      -1.686       0.586
UF_Santa Catarina                        -0.3130      0.530     -0.591      0.554      -1.351       0.725
UF_Sergipe                               -0.4260      0.648     -0.658      0.511      -1.696       0.844
UF_São Paulo                             -0.0726      0.529     -0.137      0.891      -1.110       0.965
UF_Tocantins                             -0.9692      0.683     -1.419      0.156      -2.308       0.370
Setor CNAE_COMERCIO E SERVICOS           -0.1563      0.073     -2.150      0.032      -0.299      -0.014
Setor CNAE_INDUSTRIA DE TRANSFORMAÇÃO     0.1665      0.058      2.886      0.004       0.053       0.280
Setor CNAE_INDUS

# Sem UF, com contribuição

In [5]:
import pandas as pd
import statsmodels.api as sm
import numpy as np

# Carregar os dados
tcc_data = pd.read_csv("dadosTCC_final.csv")

# Preparar variáveis dependentes e independentes
dependent_var = tcc_data['Valor Desembolsado em Reais']
df_dummy = tcc_data.drop(columns=['UF', 'Ano', 'Cliente', 'Natureza', 'Valor Desembolsado em Reais', 'Subsetor CNAE Agrupado', 'Instituição Financeira Credenciada', 'Valor da Operação em R$'])

# Criar variáveis dummy para dados categóricos
df_dummy = pd.get_dummies(df_dummy, drop_first=True)

# Remover qualquer linha com NaN e converter para tipo numérico
df_dummy = df_dummy.dropna().astype(float)  # Assegura que todos os dados são float

# Aplicar a transformação exponencial nas variáveis independentes
df_dummy = np.exp(df_dummy)  # Eleva e à potência dos valores em df_dummy

# Adicionar coluna constante para intercepto
df_dummy = sm.add_constant(df_dummy)  # Adiciona uma coluna constante para o intercepto

# Garantir que dependent_var está alinhada com df_dummy
tcc_data = tcc_data.loc[df_dummy.index]  # Alinha tcc_data com df_dummy
dependent_var = tcc_data['Valor Desembolsado em Reais']

# Aplicar a transformação logarítmica na variável dependente, garantindo que não há valores zero ou negativos
dependent_var = np.log(dependent_var + 1)  # Usando log(x + 1) para lidar com valores zero

# Resetar índices para garantir alinhamento
dependent_var = dependent_var.reset_index(drop=True)
df_dummy = df_dummy.reset_index(drop=True)

# Criar o modelo de regressão OLS
model = sm.OLS(dependent_var, df_dummy)

# Ajustar o modelo
results = model.fit()

# Calcular a mudança percentual para cada coeficiente
coefficients = pd.DataFrame({
    'coef': results.params,
    'std err': results.bse,
    't': results.tvalues,
    'P>|t|': results.pvalues,
    'conf_int_low': results.conf_int()[0],
    'conf_int_high': results.conf_int()[1]
})
coefficients['percent_change'] = (np.exp(coefficients['coef']) - 1) * 100

# Salvando os resultados em um CSV
csv_path = 'log_regTCC_com_Contribuicao_sem_UF.csv'
coefficients.to_csv(csv_path, index_label='variable')

# Exibir resumo do modelo
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:     Valor Desembolsado em Reais   R-squared:                       0.260
Model:                                     OLS   Adj. R-squared:                  0.259
Method:                          Least Squares   F-statistic:                     310.6
Date:                         Wed, 29 May 2024   Prob (F-statistic):               0.00
Time:                                 15:31:11   Log-Likelihood:                -11901.
No. Observations:                         7079   AIC:                         2.382e+04
Df Residuals:                             7070   BIC:                         2.388e+04
Df Model:                                    8                                         
Covariance Type:                     nonrobust                                         
============================================================================================================
                                               coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------
const                                       18.2764      0.386     47.328      0.000      17.519      19.033
Contribuição do UF ao PIB Nacional (%)       0.9542      0.092     10.336      0.000       0.773       1.135
Proporção do Valor Setorial ao PIB do UF  -6.83e-06   5.59e-05     -0.122      0.903      -0.000       0.000
Setor CNAE_COMERCIO E SERVICOS              -0.1371      0.072     -1.901      0.057      -0.278       0.004
Setor CNAE_INDUSTRIA DE TRANSFORMAÇÃO        0.1981      0.056      3.511      0.000       0.087       0.309
Setor CNAE_INDUSTRIA EXTRATIVA              -0.0657      0.225     -0.291      0.771      -0.508       0.376
Porte do cliente_MICRO                      -1.4364      0.110    -13.060      0.000      -1.652      -1.221
Porte do cliente_MÉDIA                      -1.0269      0.025    -41.895      0.000      -1.075      -0.979
Porte do cliente_PEQUENA                    -1.3975      0.080    -17.529      0.000      -1.554      -1.241
==============================================================================
Omnibus:                        0.826   Durbin-Watson:                   1.437
Prob(Omnibus):                  0.662   Jarque-Bera (JB):                0.853
Skew:                           0.024   Prob(JB):                        0.653
Kurtosis:                       2.977   Cond. No.                     7.63e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.63e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""